### <font color='green'>Transformers- bert for Sentiment Analysis on reviews </font> 

In [1]:
!nvidia-smi

Mon Dec 28 20:02:15 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   62C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### <font color='green'>Setup</font> 

In [1]:
!pip install sentencepiece

In [3]:
!pip install -qq transformers

     |████████████████████████████████| 1.5MB 7.9MB/s 
     |████████████████████████████████| 890kB 28.5MB/s 
     |████████████████████████████████| 2.9MB 20.2MB/s 


In [8]:
import pandas as pd
from pandas import DataFrame
import logging



In [5]:
edited_types = {
'asin':         'object',
'average'  :        'float16',
'review_body'     :  'object',
'review_date'      : 'object',
'review_likes'     : 'object',
'review_rating'    :'float16',
'review_title'     : 'object',
'five_star'        :'float16',
'four_star'        :'float16',
'one_star'        :'float16',
'pp_date'          : 'object',
'three_star'       :'float16',
'two_star'         :'float16',
'refpe'            : 'object',
'text_clean'        :'object',
'title_clean'       :'object',
'ml_score'         :'float16',
'text'             : 'object',
'ml_topic'         : 'category',
'opinion'          : 'object',
}


In [14]:
from google.colab import auth
auth.authenticate_user()

# https://cloud.google.com/resource-manager/docs/creating-managing-projects
project_id = 'dataimpact-rd'
!gcloud config set project {project_id}


Updated property [core/project].


In [15]:
# Download the file from a given Google Cloud Storage bucket.
!gsutil cp gs://di_data_sas/EN/US/Amazon/Data/periode_11/amazon_ml_opinions_topics.csv /tmp/amazon_ml_opinions_topics.csv
  
# Print the result to make sure the transfer worked.
!head -n 5 /tmp/amazon_ml_opinions_topics.csv

!gsutil cp gs://di_data_sas/EN/US/Walmart/Data/2020_periode_1/walmart_ml_opinions.csv /tmp/walmart_ml_opinions_topics.csv
!gsutil cp  gs://di_data_sas/EN/US/Target/Data/2020_periode_1/target_ml_opinions.csv /tmp/target_ml_opinions_topics.csv
!gsutil cp gs://di_data_sas/EN/UK/Asda/Data/2020_periode_1/asda_ml_opinions.csv /tmp/asda_ml_opinions_topics.csv
!gsutil cp gs://di_data_sas/EN/UK/Morrisons/Data/2020_periode_1/morrisons_ml_opinions.csv /tmp/morrisons_ml_opinions_topics.csv
!gsutil cp gs://di_data_sas/EN/UK/Ocado/Data/2020_periode_1/ocado_ml_opinions.csv  /tmp/ocado_ml_opinions_topics.csv
!gsutil cp gs://di_data_sas/FR/Coursesu/Data/2020_periode_10/coursesu_ml_opinions.csv courseu.csv

Copying gs://di_data_sas/EN/US/Amazon/Data/periode_11/amazon_ml_opinions_topics.csv...
/ [1 files][  1.7 GiB/  1.7 GiB]   48.0 MiB/s                                   
Operation completed over 1 objects/1.7 GiB.                                      
asin,average,review_body,review_date,review_likes,review_rating,review_title,five_star,four_star,one_star,pp_date,three_star,two_star,refpe,text_clean,title_clean,ml_score,text,ml_topic,opinion
B071F4PVKJ,0.0,the seam on the left sleeve broke in 2 days.  it was likely just an anomaly because the knitting is very high quality otherwise.,2019-01-04,0,2.0,"overall nice, but mine had a bad seam",,,,2019-11-31,,,1840002706176,seam left sleeve broke days likely anomaly knitting high quality otherwise,overall nice mine bad seam,-1.0,seam left sleeve broke days likely anomaly knitting high quality otherwise overall nice mine bad seam,['delivery'],['']
B071F4PVKJ,0.0,loved everything about this sweater..it looks adorable on my dog and it was everyth

In [16]:
list_retailers = ['Amazon' , 'Asda', 'Morrisons', 'Ocado', 'Target', 'Walmart']
retailers = {}
for retailer in list_retailers:

    retailers[retailer] = pd.read_csv('/tmp/' + retailer.lower() + 
                                      '_ml_opinions_topics.csv', dtype=edited_types, nrows=10)
    #retailers[retailer] = retailers[retailer].dropna(subset=['review_body'])
    
to_concat = [retailers[retailer] for retailer in list_retailers]
#to_concat = [retailers[retailer]['review_body'] for retailer in list_retailers]
data = pd.concat(to_concat, ignore_index = True)

### <font color='green'>Handler class, handle get, tokenization and padding of the sentences for Pytorch Dataset  </font> 

In [2]:
import pickle
topics =  pickle.load(open('topics.p', 'rb'), encoding='latin1')
targets =  pickle.load(open('targets.p', 'rb'), encoding='latin1')
targets_en = targets + ['quality']

In [3]:

# pose sequence as a NLI premise and label as a hypothesis
from transformers import AutoModelForSequenceClassification, AutoTokenizer
model = 'joeddav/xlm-roberta-large-xnli'
nli_model = AutoModelForSequenceClassification.from_pretrained(model)
tokenizer = AutoTokenizer.from_pretrained(model)
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Some weights of the model checkpoint at joeddav/xlm-roberta-large-xnli were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
targets_fr = ['competition', 'prix', 'goût', 'livraison', 'qualité']
hypothesis = "Cet exemple est {}."
hypothesis = "This exemple is {}."
results = {}

sequence = 'pas assez moelleux à mon goût mais bons quand même pruneaux un peu secs trop cher par contre '
sequence = 'délicieux, bio et d\'un pris raisonnable pour un 40% de cacao... bonne alternative gourmande'
sequence  = 'produit délicieux, facile à cuisiné (j\'ai décongelé 1 nuit dans du lait) bon produit'
for label in targets_fr:
  # run through model pre-trained on MNLI
  print(hypothesis.format(label))

  x = tokenizer.encode(sequence, hypothesis.format(label), return_tensors='pt',
                      truncation=True)
  logits = nli_model(x)[0]

  # we throw away "neutral" (dim 1) and take the probability of
  # "entailment" (2) as the probability of the label being true 
  entail_contradiction_logits = logits[:,[0,2]]
  probs = entail_contradiction_logits.softmax(dim=1)
  prob_label_is_true = probs[:,1]
  results[label] = (prob_label_is_true[0])


This exemple is competition.
This exemple is prix.
This exemple is goût.
This exemple is livraison.
This exemple is qualité.


In [6]:

labels_list = {k:v.item() for k,v in results.items()}
labels_list
predicted_rating = max(labels_list, key=labels_list.get)
predicted_rating
{k: v for k, v in labels_list.items() if v > 0.6 and  labels_list[max(labels_list, key=labels_list.get)]  / v < 1.2 }
labels_list

{'competition': 0.032359931617975235,
 'goût': 0.9963603615760803,
 'livraison': 0.5836494565010071,
 'prix': 0.06994394958019257,
 'qualité': 0.9971575736999512}

In [55]:
  def inference(sequence):

    results = {}
    for label in targets:
      # run through model pre-trained on MNLI

      x = tokenizer.encode(sequence, hypothesis.format(label), return_tensors='pt',
                          truncation=True)
      logits = nli_model(x)[0]

      # we throw away "neutral" (dim 1) and take the probability of
      # "entailment" (2) as the probability of the label being true 
      entail_contradiction_logits = logits[:,[0,2]]
      probs = entail_contradiction_logits.softmax(dim=1)
      prob_label_is_true = probs[:,1]
      results[label] = (prob_label_is_true[0])
  
      # getting the proba value as torch item
    results = {k:v.item() for k,v in results.items()}
    max_prob = max(results, key=results.get)

    results = {k: v for k, v in results.items() if v > 0.6 and  results[max_prob]  / v < 1.2 }
    return results
inference('got shipped fast thx but its good product')

{'delivery': 0.9922468662261963,
 'quality': 0.9988862872123718,
 'taste': 0.903903067111969}

In [15]:

class topics_custom_pipeline():

  """Custom class that use an Natural Language Inference Model and the right encoder to
  Extract Main Topics from Reviews"""
  def __init__(self, nli_model, tokenizer, targets,df: DataFrame, lang='EN'):
    self.nli_model = nli_model
    self.tokenizer = tokenizer
    self.targets = targets
    self.df = df
    self.lang = lang


  def select_hypothesis(self):
    _hypothesis = { 'FR': "Cet exemple est {}.",
                    'EN': "This exemple is {}."}
    
    try : return _hypothesis[self.lang]
    except:
        logging.error('Default Language English is chosen, beceuse you didnt specify a right language [EN, FR, ES..]')
        return _hypothesis['EN']


  def inference(self, sequence):

    results = {}
    for label in self.targets:
      # run through model pre-trained on MNLI
      x = self.tokenizer.encode(sequence, self.select_hypothesis().format(label), return_tensors='pt',
                          truncation=True)
      logits = self.nli_model(x)[0]

      # we throw away "neutral" (dim 1) and take the probability of
      # "entailment" (2) as the probability of the label being true 
      entail_contradiction_logits = logits[:,[0,2]]
      probs = entail_contradiction_logits.softmax(dim=1)
      prob_label_is_true = probs[:,1]
      results[label] = (prob_label_is_true[0])

      # getting the proba value as torch item
    results = {k:v.item() for k,v in results.items()}
    # results['taste'] = results['taste'] - 0.01
    add_competition = False
    max_prob = max(results, key=results.get)
    results = {k: v for k, v in results.items() if v > 0.85 and  results[max_prob]  / v < 1.2 }
    if max_prob == 'quality' or max_prob == 'qualité':
      if 'taste' in results.keys():
        if results['taste'] < 0.995:
          results = list(filter(lambda x: x not in 'taste',([key for key in results.keys()])))
      elif 'goût' in results.keys():
        if results['goût'] < 0.995:
          results = list(filter(lambda x: x not in 'goût',([key for key in results.keys()])))
    results = list(filter(lambda x: x not in 'quality' and x not in  'qualité', results))
    return results
   

  def extract_topics_on_df(self):

    self.df['topics'] = self.df['review'].apply(lambda x: self.inference(x))
    return self.df


In [16]:
take = pd.read_csv('takealot_final_ml_processed.csv')

courseu = pd.read_csv('courseu.csv')

In [21]:
sample_fr = courseu.sample(10)
sample_en = take.sample(10)

extraction_object_fr = topics_custom_pipeline(nli_model, tokenizer, ['competition', 'prix', 'goût', 'livraison', 'qualité'], sample_fr, 'EN')
extraction_object_en = topics_custom_pipeline(nli_model, tokenizer, targets_en, sample_en)

In [22]:
data_processed_fr = extraction_object_fr.extract_topics_on_df()
data1_processed_en = extraction_object_en.extract_topics_on_df()

In [23]:
pd.set_option('display.max_colwidth', -1)
data_processed_fr[['review', 'topics']].style.applymap(lambda x: 'background-color:pink; border:solid; color:black; font: italic 1.2em "Fira Sans", serif;' )

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


,review,topics
10400,rarement déçue. aussi bonnes en fruits d appoint que pour réaliser des tartes. achetées régulièrement,[]
7637,j achète ce produit régulièrement j en suis satisfaite aussi bien que les produits grandes marques,[]
5499,"très bon chocolat, pour les pauses thé/café. prix très abordable pour du bio. chocolat bio très abordable","['competition', 'prix', 'goût']"
1243,"ma fille a longtemps cherché ce produit. enfin elle est comblée, car ses cheveux sont impeccables. très efficace et pratique",[]
1279,"j'achète ce produit depuis des années pour mes enfants, à mon goût il est trop sucré mais eux n'aiment que lui brique multi vitaminé",['goût']
1960,a éviter l'hiver de octobre à mars les chèvres allaitant leurs petits produit de qualité.,[]
9675,je l achete depuis plusieurs années. c est parfait. ideal la matiere végétale pour les chats,[]
31719,"ces galettes sont très bonnes. elles sont plus petites que les autres, ce qui est bien pratique : elles ne débordent pas de la poêle. très bonnes",[]
35113,"bon produit!! des tranches de jambon fraîches, rien de comparable à un bon jambon à l’os, mais on saitce qu’on achète, et la qualité est là.. c’est l’essentiel bon rapport qualité/ prix",['prix']
4661,je n'utilise que ce produit depuis des mois et j'adore produit excellent et pas cher,"['prix', 'goût']"


In [24]:
pd.set_option('display.max_colwidth', -1)
data1_processed_en[['review', 'topics']].style.applymap(lambda x: 'background-color:pink; border:solid; color:black; font: italic 1.2em "Fira Sans", serif;' )

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


,review,topics
1386,we love these starbucks pods.,['packaging']
2456,"i didn't like it. the smell and the taste, it's a no from me. i prefer the white one thank you.","['allergens', 'competition', 'taste']"
264,this coffee is absolutely awful. what a bad buy,"['packaging', 'price']"
907,great tasting coffee at a good price,"['price', 'taste']"
1751,not enough flavour,['taste']
2683,others like wine i like coffee. this is truely a taste for all who loves coffee!!!,['taste']
6,very nice product and nice to have a variety,[]
2723,i can't choose a favourite between guatemala and columbia :),['competition']
669,a little bit slow in machine otherwise it is perfect,[]
2578,not a fan of the coffee personally but my family loves it!,[]


In [24]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification", model='joeddav/xlm-roberta-large-xnli')
sequence = 'très satisfaite . produit de bonne qualité et beaucoup de saveurs délicieux'
candidate_labels = targets_fr
classifier(sequence, candidate_labels, multi_class=True)